# P1. Regression

In [205]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt

# data 읽어오기
columns = ['A', 'B', 'C', 'D']
df = pd.read_csv('P1 regression.csv', header=None)
df.columns = columns
df.head()

,A,B,C,D
0,1.385735,2.768262,144.080816,139.249071
1,1.003630,0.999884,0.642313,0.989770
2,1.242363,1.937306,28.660409,23.324634
3,1.128282,1.542643,27.225159,11.382374
4,0.913687,0.732714,0.188189,0.237336


In [206]:
# data 정규화 작업
df = (df - df.mean()) / df.std()

df.head()

,A,B,C,D
0,3.118012,4.269326,12.647311,1.610438
1,-0.327532,-0.431385,-0.415782,-0.154529
2,1.825184,2.060475,2.135855,0.130589
3,0.796483,1.011378,2.005145,-0.021861
4,-1.138574,-1.141578,-0.457140,-0.164135


In [207]:
# 변수 구분하기
X = df[['A', 'B', 'C']] # 세가지 독립변수 A, B, C
y = df['D'] # 종속변수 D


# linear regression 모델 선언
regr = linear_model.LinearRegression()

# regression 학습
regr.fit(X, y)

print('절편 = ', regr.intercept_)
print('기울기 = ', regr.coef_)

절편 =  2.506582105564225e-15
기울기 =  [-0.46758639  0.5655295   0.22412999]


In [208]:
prediction = regr.predict(X)

# 모델 평가하기
# mean squared error
mse = np.sum((prediction - y)**2)
# root mean squared error
rmse = np.sqrt(mse/(len(y)))

# sum of square of residuals
ssr = np.sum((prediction - y)**2)
#  total sum of squares
sst = np.sum((y - np.mean(y))**2)
# R2 score
r2  = 1 - (ssr/sst)

# printing values
print('Root mean squared error: ', rmse)
print('R2 score: ', r2)



Root mean squared error:  0.9424000492448669
R2 score:  0.11099314032359531


# P2. Growth - Sales/Earning

### Sales Growth Formula

매출액의 과대 또는 과소계상 효과를 제거하고 정상 매출 활동으로 벌어들인 순매출을 산정하기 위해 매출채권(t, t-1)과 선수금(t, t-1)을 Sales(t), Sales(t-1)에서 각각 차감

(Sales(t) - account_receivable(t) - advanced_payment(t)) / (Sales(t-1) - account_receivable(t-1) - advanced_payment(t-1)) - 1

### Earnings Growth Formula

비현금성수익 및 비용의 효과를 제거하고 영업활동과 관련된 이익만 계상하기 위해 감가상각비(t, t-1), 순매입채무증감(t, t-1)을 더하고, 순매출채권증감(t, t-1), 순재고자산증감(t, t-1)을 차감

(Earnings(t) + depreciation(t)) - net_account_receivable(t) - net_inventory(t) + net_account_payable(t) / (Earnings(t-1) + depreciation(t-1) - net_account_receivable(t-1) - net_inventory(t-1) + net_account_payable(t)) - 1

# P3. Technical indicator - trend-line

daily price curve의 국소 최소값의 개수를 n이라고 할 경우, 선형보간법(linear interpolation)을 사용해 지지선(linear)를 도출할 수 있다. n == 2 경우 두 점을 이은 하나의 선형 지지선이 되지만, 지지선이 선형(linear)이므로 n >= 3일 경우 특정 local minima가 지지선에 접하지 않는 경우가 발생한다. daily price curve는 비선형이고 균등하지 않을 확률이 높으므로 모든 국소 최소값에 지지선을 맞춰 도출할 경우 지지선의 선형성(linearity)이 없어지거나, 고점 대비 낙폭이 적은 국소 최소값에 접하는 지지선을 상정할 경우 기타 국소 최소값을 침범해 지지선이 해당 최소값보다 높아지는 경우가 발생한다. 따라서 모든 국소 최소값이 지지선에 접하지 않을 수 있다.

### 1. 국소최소값 찾기

In [152]:
def find_local_minima(prices): # 특정 기간의 sequencial 가격 array == prices
    minima = {}
    length = len(prices)
    if length >= 2: # 전체 가격 개수가 2 이상일 경우,
        if prices[0] < prices[1]: # 첫번째 값이 두번째 값보다 낮다면
            minima[0] = prices[0] # minima에 해당 시점(0)을 key로, 해당 가격을 value로 담기
            
    if length > 3: # 전체 가격 개수가 3 초과일 경우,
        for i in range(1, length-1): # prices의 두번째 값에서 마지막에서 두번째 값까지
            if prices[i] < prices[i-1] and prices[i] < prices[i+1]: # 해당 시점 가격이 이전 시점 대비 & 다음 시점 대비 낮다면
                minima[i] = prices[i] # minima에 해당 시점(i)을 key로, 해당 가격을 value로 담기
                
    if prices[length-1] < prices[length-2]: # prices의 마지막 시점 가격이 바로 이전 시점 가격보다 낮다면
        minima[length-1] = prices[length-1] # minima에 해당 시점(length-1)을 key로, 해당 가격을 value로 담기
        
    return minima # 국소최소값이 담긴 dictionary 반환
        

In [164]:
prices = [20, 17, 16, 12, 14, 17, 23, 27, 29, 31, 27, 14, 11, 14, 17]

t = list(find_local_minima(prices).keys())
p = list(find_local_minima(prices).values())

### 2. 지지선의 선형함수 계산

##### 2-1. 국소최소값이 2개일 경우

In [203]:
def calc_support(x):
    # (y1 - y2) / (x1 - x2) == 기울기 계산식
    slope = (p[1] - p[0]) / (t[1] - t[0])

    # 절편 계산식(선형보간법 보간함수)
    # t == 0 일때 p값 = intercept 
    intercept = p[0] - (slope*t[0])
    
    # 특정 시점(t)에 대한 지지선 위치(s)
    support_line = slope*x + intercept
    print("기울기=", round(slope, 2))
    print("절편=", round(intercept, 2))
    return support_line

In [204]:
calc_support(3)

기울기= -0.11
절편= 12.33


12.0

##### 2-2. 국소최소값이 3개 이상일 경우

In [178]:
import numpy as np
from sklearn import linear_model

In [200]:
# 변수 구분하기
X = np.array(t)
y = np.array(p)

# linear regression 모델 선언
regr = linear_model.LinearRegression()

# regression 학습
regr.fit(X, y)

print("기울기=", regr.intercept_)
print("절편=", regr.coef_)

# M1. Shortest Path

In [1]:
# 1차 단계(대략적인 최소값 pair 찾기)
# 수직터치해야 하는 수평선상 임의의 점 C와 점A에서 수평선으로 수선의 발을 접하는 점 A'를 꼭짓점으로 하는 ACA' 삼각형 상정
# 점C와 점B에서 수평선으로 수선의 발을 접하는 점 B'를 꼭짓점으로 하는 BCB' 삼각형을 상정했을 때,
# 수평선을 임의로 나눠 선A'C, 선CB'의 길이를 구한 후 피타고라스 정리를 사용해 선AC, 선CB를 각각 구한 후 가장 최솟값이 되는 합을 구한다.
# 0 ~ 10 사이의 정수로 먼저 케이스를 대입해보며 최소값에 근사한 값을 나타내는 점 C의 위치를 가늠한다.

# A'C 사이의 거리를 변수 X라고 했을 때,
X = range(0, 10) # 0 ~ 10 사이의 거리 케이스
distances = {} # 최종 ACB 거리를 key로, (A'C, CB') 페어를 value로 하는 딕셔너리

for x in X:
    y = (10-x) # CB' 사이의 거리를 변수 y라고 했을 때,
    dist = ((8**2) + (x**2))**(0.5) + ((6**2) + (y**2))**(0.5) # 피타고라스 정리를 사용한 최종 길이를 변수 dist라고 한다.
    distances[dist] = (x, y) # distances 딕셔너리에 값을 넣는다.
    
print(distances)
min_dist = min(distances.keys()) # 가장 최솟값을 가지는 최종 길이
print(min_dist, distances[min_dist]) # 최소 길이 값, (A'C, CB') 페어

{19.6619037896906: (0, 10), 18.87891157469052: (1, 9), 18.246211251235323: (2, 8), 17.763548202610416: (3, 7), 17.429553284237727: (4, 6), 17.244230807963255: (5, 5), 17.21110255092798: (6, 4), 17.33834974523402: (7, 3), 17.63826381932152: (8, 2), 18.124357109090514: (9, 1)}
17.21110255092798 (6, 4)


In [2]:
# 2차 단계(상세 값 찾기)
# 선 AA', 선 BB'의 비율이 8:6의 비율이며,
# 1차 단계에서 도출한 정수 범위 한정 최솟값 pair도 이와 비슷한 비율인 점을 토대로 유추해볼 때,
# 최소 길이를 도출하는 (A'C, CB') 페어역시 8:6의 비율로 구성되어 있는 경우 상정

x = 10*(8/14) # 수평선 10m 중 8의 비율을 가지는 A'C의 길이
y = 10*(6/14) # 수평선 10m 중 6의 비율을 가지는 CB'의 길이

x_ = ((8**2) + x**2)**(0.5) # 피타고라스 정리를 사용한 AC 길이
y_ = ((6**2) + y**2)**(0.5) # 피타고라스 정리를 사용한 CB 길이

print(x_ + y_) # 최종적으로 도출한 길이 최솟값
print(x, y) # 최종길이 최솟값을 도출하는 (A'C, CB') 페어

17.204650534085253
5.7142857142857135 4.285714285714286


# M5. Poker Game

In [134]:
count_possible = 20 * 19
num_possible = []
for i in range(1, 11):
    num_possible.append(i)
num_possible = num_possible[::-1]

# consecutive numbers list
consec = []
for n, i in enumerate(num_possible):
    pair = (num_possible[n], num_possible[n-1])
    consec.append(pair)
    
consec

In [135]:
# pairs
pairs = []
for i in num_possible:
    pair = (i, i)
    pairs.append(pair)
    
pairs

In [136]:
# high numbers
highs = []
for i in num_possible:
    for j in num_possible:
        pair = (i, j)
        if (pair not in consec) and (pair not in pairs) and (pair[::-1] not in consec) and (pair[::-1] not in pairs):
            highs.append(pair)
            
highs

In [45]:
total_outcome = consec + pairs + highs

In [137]:
total_outcome

# E1. Discounted Cash Flow problem

In [54]:
rent_y1 = 10000.00 # 1년 뒤 첫 월세
total_year = 5 # 총 계약 기간
interest_rate = 0.03 # 2년 뒤 이자율
inflation_rate = 0.02 # 2년 위 물가상승률

# 2년마다 조정되는 월세 계산
rent_y2 = rent_y1 * ((1 + inflation_rate)**2)
rent_y3 = rent_y2
rent_y4 = rent_y3 * ((1 + inflation_rate)**2)
rent_y5 = rent_y4
rents = [rent_y1, rent_y2, rent_y3, rent_y4, rent_y5]

# 단리 기반 현재가치 계산(PV = CFn / (1 + nr))
# 2년 뒤부터 이자율이 3%이므로 1년 뒤는 이자율 0%(1년 뒤 미래가치는 현재가치와 동일)
# 따라서 2년 뒤부터 5년 뒤까지의 4번의 현금흐름에 대해 단리 기반 현재가치 계산 후 1년 뒤 현재가치 합산
pv_after_y2 = sum(rents[1:]) / (1+(4*interest_rate))
pv = pv_after_y2 + rent_y1
print(pv)

47907.71714285714


# E2. Term Structure of Interest Rates

1) If Term structure of interest rates is upward sloping, fill this relationship (YTM, Spot Rate,
Forward Rate)

먼저, 채권에 붙어있는 표면이자율인 Par rate을 사용해 Spot Rate를 도출할 수 있다. n년간(6개월마다 이자를 지급할 경우 0.5년마다) n * 2번 (원금 x Par rate)만큼 이자를 지급할 경우, 해당 이자 지급일 별 cash flow(만기 때는 원금 포함)를 '만기까지 이자 지급이 없는 할인채'로 가정해 현재가지로 할인할 수 있다. 이때 사용하는 할인율이 Spot rate다.

YTM은 채권에서 발생하는 미래 현금흐름을 현재가치로 할인했을 때 현재 채권의 가격과 일치시켜주는 할인율이다. 우상향하는 Yield curve에서는 Spot rate가 YTM보다 크다.

Forward rate는 Spot rate에 내재되어 있는 미래 이자율이다. 구체적으로는 longterm 채권을 사서 만기까지 보유했을 때의 수익률과 shorterm 채권을 사고 roll-over했을 때의 수익률을 동일하게 만들어주는 이자율이 Forward rate다. 우상향하는 Yield curve에서는 Forward rate가 Spot rate보다 크다.

2) I have a treasure bond that maturity is 3Y, face value 1000, coupon rate 5%, annual coupon Calculate bond price

1000 = 1025 / (1 + x)

In [62]:
face_value = 1000
coupon_rate = 0.05
coupon = face_value * coupon_rate / 2

# case 1
rates = 0.03
year = 1
n = year * 2

total_pv = 0
for i in range(1, n+1):
    pv = (coupon / ((1 + (rates/2))**i))
    total_pv += pv
total_pv += (face_value / ((1 + (rates/2))**n))
print(total_pv)

1019.5588342352402


In [63]:
# case 2
rates = 0.04
year = 2
n = year * 2

total_pv = 0
for i in range(1, n+1):
    pv = (coupon / ((1 + (rates/2))**i))
    total_pv += pv
total_pv += (face_value / ((1 + (rates/2))**n))
print(total_pv)

1019.0386434933715


In [64]:
# case 3
rates = 0.05
year = 3
n = year * 2

total_pv = 0
for i in range(1, n+1):
    pv = (coupon / ((1 + (rates/2))**i))
    total_pv += pv
total_pv += (face_value / ((1 + (rates/2))**n))
print(total_pv)

1000.0000000000005


# C1. The second most frequent vowel letter

In [132]:
def check_vow(string, vowels='aeiou'): 
    
    string = string.lower() # 모든 string을 소문자로 변환

    # 각 vowel를 key로, 0을 value로 가지는 count dictionary
    count = {}.fromkeys(vowels, 0) 
      
    # vowel 카운팅
    for char in string: # 주어진 string의 각 글자별로
        if char in count: # count dictionary에 포함되어 있는 string이라면
            count[char] += 1 # count dict에서 해당 vowel 1씩 count
    return count # 최종 coutn dict 반환

In [133]:
# test
string = "abcdeeeooauiea" # 예시 string
count_dict = check_vow(string) # check_vow 함수로 vowel 카운팅
value = list(sorted(count_dict.values()))[-2] # 오름차순으로 정렬한 다음 뒤에서 두번째 횟수(두번째 높은 횟수) 찾기
for key, value2 in count_dict.items(): # count_dict에서 key, value 중
    if value2 == value: # 두번째 높은 횟수(value)를 찾아
        print(key) # 해당 vowel(key) 반환

a


# C3. MDD

In [44]:
# time과 price sequence list
t_price_dict = {0:10, 1:11, 2:12, 3:13, 4:12, 5:10, 
                6:9, 7:13, 8:14, 9:15, 10:18, 11:19,
               12:20, 13:22, 14:23, 15:20, 16:17, 17:16,
                18:19, 19:24, 20:23}

price_list = list(t_price_dict.values())

In [45]:
def find_mdd_start(price_list):
    # MDD가 시작하는 구간 찾기
    down_list = []
    for t, p in enumerate(price_list[:-1]):
        # 이전 시간(t-1) 가격보다 낮고, 다음 시간(t+1) 가격보다 높은 현재 가격 찾기 == 가격 하락 구간
         if (price_list[t-1]) > p and (p > price_list[t+1]):
                down_list.append(t-1) # MDD 가격 하락 구간

    # 가격 하락 구간 중 MDD 시작 시점 찾기
    for n, l in enumerate(down_list):
        # sequence 중 기간(n)이 1 차이씩 연속되는 구간 중 첫번째 t만 남기고 삭제 == MDD 시작 시점만 남기기
        if l-1 == down_list[n-1]:
            del down_list[n]
            
    return down_list

In [46]:
def find_mdd_recover(start_list, price_list):
    # MDD 시작 가격 회복 시점 찾기
    recover_n = []

    for loop in range(len(start_list)):
        n_start = start_list[loop] # MDD 시작 시점마다,
        for n, p in enumerate(price_list):
            # MDD 시작 시점 이후 & MDD 시작 시점 가격보다 크거나 같은 가격 찾은 후,
            if (n > n_start) and (price_list[n_start] <= p):
                recover_n.append(n) # recover_n에 담기
                break # 1회만 찾고 break(회복 시점은 단 한 번)
                
    return recover_n

In [47]:
def find_lowest_price(start_list, recover_list, price_list):
    # 각 MDD 별 가장 낮은 가격 찾기
    lowest_prices = []

    for i in range(len(recover_n)): # MDD 개수 별로,
        start = start_list[i] # MDD 시작 시점
        end = recover_list[i] + 1 # MDD 완전 회복 시점
        p_list = []
        for j in range(start, end): # 시작 시점에서 회복 시점 사이에서,
            p = price_list[j] # 각 시점별 가격을 
            p_list.append(p) # p_list에 담고
        lowest_p = min(p_list) # 그중 가장 낮았던 가격만 
        lowest_prices.append(lowest_p) # lowest_prices에 담는다

    return lowest_prices

In [48]:
def calc_mdd_size(start_list, price_list, lowest_prices):
    # 각 MDD size 계산하기
    mdd_sizes = []
    for i in range(len(start_list)): # MDD 개수 별로,
        start = start_list[i] # MDD 시작 시점
        start_price = price_list[start] # MDD 시작 시점 가격
        lowest_price = lowest_prices[i] # 가장 낮았던 가격
        mdd_size = start_price - lowest_price # MDD size를
        mdd_sizes.append(mdd_size) # mdd_sizes에 담기

    return mdd_sizes

In [49]:
def result_matching(mdd_sizes, recover_list):
    # MDD size와 회복 시점 tuple로 매칭
    result_pairs = []
    for i in range(len(mdd_sizes)): # MDD 개수 별로,
        mdd_size = mdd_sizes[i] # MDD size
        recover_t = recover_list[i] # MDD 회복시점
        pair = (mdd_size, recover_t) # pair tuple 생성 후
        result_pairs.append(pair) # result_paris에 담기

    result_pairs.sort(reverse=True) # mdd_size가 큰 순서대로 정렬
    return result_pairs

In [50]:
def mdd_main(k):
    start_list = find_mdd_start(price_list)
    recover_list = find_mdd_recover(start_list, price_list)
    lowest_prices = find_lowest_price(start_list, recover_list, price_list)
    mdd_sizes = calc_mdd_size(start_list, price_list, lowest_prices)
    result_pairs = result_matching(mdd_sizes, recover_list)
    
    print(result_pairs[:k]) # k 개수만큼 출력

In [51]:
mdd_main(1)

[(7, 19)]


# C102

In [ ]:
# 시도했으나 작동하지 않는 코드
# 작동하는 코드는 다음 셀부터

filename = 'Email Coding C102.txt'
f = open(filename, 'r+')

X_matrix = []
for i in range(200):
    X_row = []
    for j in range(200):
        sum_elem = 0
        for k in range(10):
            a_matrix = f.read().split('|')[k].replace("float('nan')", "0") # from 0 to 9 matrices ("float('nan')") to 0
            row_by_row = a_matrix.split(';') # len == 200 rows
            a_row = [float(num) for num in row_by_row[i].split(',') if num != 'nan'] # from 0 to 199
            elem = a_row[j] # from 0 to 199
            sum_elem += elem
            
        mean_elem = sum_elem / 10
        X_row.append(mean_elem)
    X_matrix.append(X_row)
    
print(X_matrix)
f.close()

## mean(X) Matrix & median(Y) Matrix

In [124]:
from tqdm import tqdm # for progress bar

In [90]:
# file read 후 각 matrix 변수 담기
# 본래 위 코드와 같이 for loop를 사용해 각 Matrix를 읽어오려 했으나 대용량 파일 read 이슈로 인해 각각 따로 read
filename = 'Email Coding C102.txt'
f = open(filename, 'r+')

# 원본 데이터에 포함된 nan, inf 값을 0으로, \n을 제거하는 함수
def replace_string(matrix):
    chars = ["float('nan')", "float('inf')"]
    for char in chars:
        matrix = matrix.replace(char, "0")
    matrix = matrix.replace("\n", '')
    return matrix

# 아래부터 10개 각 Matrix read
f = open(filename, 'r+')
first_matrix = f.read().split('|')[0]
first_matrix = replace_string(first_matrix)
f.close()

f = open(filename, 'r+')
second_matrix = f.read().split('|')[1]
second_matrix = replace_string(second_matrix)
f.close()

f = open(filename, 'r+')
third_matrix = f.read().split('|')[2]
third_matrix = replace_string(third_matrix)
f.close()

f = open(filename, 'r+')
forth_matrix = f.read().split('|')[3]
forth_matrix = replace_string(forth_matrix)
f.close()

f = open(filename, 'r+')
fifth_matrix = f.read().split('|')[4]
fifth_matrix = replace_string(fifth_matrix)
f.close()

f = open(filename, 'r+')
sixth_matrix = f.read().split('|')[5]
sixth_matrix = replace_string(sixth_matrix)
f.close()

f = open(filename, 'r+')
seventh_matrix = f.read().split('|')[6]
seventh_matrix = replace_string(seventh_matrix)
f.close()

f = open(filename, 'r+')
eighth_matrix = f.read().split('|')[7]
eighth_matrix = replace_string(eighth_matrix)
f.close()

f = open(filename, 'r+')
nineth_matrix = f.read().split('|')[8]
nineth_matrix = replace_string(nineth_matrix)
f.close()

f = open(filename, 'r+')
tenth_matrix = f.read().split('|')[9]
tenth_matrix = replace_string(tenth_matrix)
f.close()


In [95]:
# 전체 Matrix가 담긴 list
matrices = [first_matrix, second_matrix, third_matrix, forth_matrix, fifth_matrix,
            sixth_matrix, seventh_matrix, eighth_matrix, nineth_matrix, tenth_matrix]

num_matrices = len(matrices) # 총 Matix 개수 == 10

In [119]:
# Median 값을 구하기 위한 함수
def median(data):
    data.sort()
    mid = len(data) // 2
    return (data[mid] + data[~mid]) / 2

In [125]:
# X Matrix를 계산하기 위해 총 10개 Matrix의 각 200개 row의 각 200 elem을 순회하며 계산

X_matrix = []
Y_matrix = []
for i in tqdm(range(200)):
    X_row = []
    Y_row = []
    for j in range(200):
        sum_elem = 0 # 평균 계산을 위한 총합 for Matrix X
        list_elem = [] # median 계산을 위한 list for Matrix Y
        for matrix in matrices: # 모든 Matrix를 하나씩 반복하며
            row_by_row = matrix.split(';') # len == 200 rows인 matrix
            a_row = [float(num) for num in row_by_row[i].split(',')] # 각 i번째 row별로 (from 0 to 199)
            elem = a_row[j] # i 번째 row의 j 번째 element
            sum_elem += elem # 각 Matrix [i,j]의 총합 for Matrix X
            list_elem.append(elem) # 각 Matrix [i,j] elem 모음 for Matrix Y
            
        mean_elem = sum_elem / num_matrices # Marix X의 element인 평균값을 구한 후
        X_row.append(mean_elem) # X_row에 담기
        
        median_elem = median(list_elem) # Matrix Y의 element인 median을 구한 후
        Y_row.append(median_elem) # Y_row에 담기
        
    X_matrix.append(X_row) # 200개로 이루어진 X_row가 모두 담기면 최종 X_matrix에 담기
    Y_matrix.append(Y_row) # 200개로 이루어진 Y_row가 모두 담기면 최종 Y_matrix에 담기
    